In [8]:
import pandas as pd
from itertools import cycle
import gpflow
import numpy as np
from gpflow.utilities import print_summary, positive
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import statsmodels.api as sm
import dill as pickle

sns.set(style="white")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_summary_fmt("notebook")

<ipython-input-8-06da3225ef62>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
final_data = pd.read_pickle("tmp/final_data.pkl")
u0 = pd.read_pickle("tmp/u0.pkl")
linear_filters_states = pickle.load(open('tmp/linear_filters_states.pkl', 'rb'))
descriptive_stats = pd.read_pickle("tmp/residual_descriptive_stats.pkl")

In [10]:
len(linear_filters_states)

17

In [3]:
ORDER = len(list(linear_filters_states.values())[0][0])
N = len(linear_filters_states)

NameError: name 'linear_filters_states' is not defined

## Rhos

In [7]:
results_states.keys()

dict_keys(['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas'])

In [ ]:

state_residual = results_states["Alabama"][1]
ORDER = int(descriptive_stats.loc[descriptive_stats.state=="Alabama", "optimal lag"])
rho_, sigma_ = sm.regression.yule_walker(state_residual, order=ORDER, method="mle")
len

In [11]:
for i, (hX, epsX) in enumerate(linear_filters_states.values()):
    print(i, hX)

0 [-0.20146002  0.15406379  0.57237811]
1 [-0.22278488  0.04580468  0.01233745 -0.18271465  0.26658548 -0.25023515
  0.74198619]
2 [-0.21834862  0.07211645  0.39587389]
3 [0.16727922]
4 [0.43604558]
5 [-0.27697628  0.18866375  0.07644872 -0.32963489  0.7205469 ]
6 [-0.15439855 -0.09507318 -0.14010959 -0.05250449 -0.06288609  0.33904084]
7 [-0.18118596  0.08080095 -0.18567735  0.10321467  0.27770449]
8 [0.11855228 0.61669982]
9 [-0.15144291 -0.11152191  0.02694584 -0.13236132  0.37332853]
10 [-0.17916254  0.09956574  0.38960707]
11 [-0.18129723  0.52955387]
12 [-0.26272272  0.13068273 -0.01824872 -0.27199511  0.62690492]
13 [-0.23442645  0.51582658]
14 [-0.19123557  0.05341087 -0.1232227   0.21293816  0.21019968]
15 [0.26483504]
16 [-0.15782528  0.132635    0.30568647]


In [16]:
-0.15782528*-0.15782528

0.024908819007078403

In [12]:
np.convolve(hX, hX[:-1], mode="same")

array([ 0.02490882, -0.04186631, -0.03065301])

In [13]:
np.convolve(hX, hX, mode="same")

array([-0.04186631, -0.07889806,  0.08108945])

In [3]:
times = None
for s in final_data.index.unique():
    s_times = set(final_data.loc[s, 'Updated'].unique())
    if times is None:
        times = s_times
    else:
        times = times.intersection(s_times)

In [5]:
u0_ = u0[u0['Updated'].isin(times)]['Residual'].values
T = u0_.shape[0]
t = np.arange(1, T + 1).reshape(-1, 1)

T2 = T - ORDER + 1

rhos = np.zeros((T2*N, T2*N))
for i, (hX, epsX) in enumerate(linear_filters_states.values()):
    for j, (hY, epsY) in enumerate(linear_filters_states.values()):
        convol = np.convolve(hX, hY[::-1], mode="same")
        for t in range(T2):
            data = convol[max(len(convol)//2 - t, 0): min(len(convol), T2 - t + len(convol)//2)]
            jstart = j*T2+max(t - len(convol)//2, 0)
            jend = jstart + len(data)
            rhos[i*T2+t, jstart:jend] = data

In [6]:
# validation
for x in range(N):
    for y in range(N):
        for t in range(T2):
            for s in range(T2):
                assert rhos[x*T2+t, y*T2+s] == rhos[y*T2+s, x*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{XY}}_{{t,s}} = \rho^{{YX}}_{{s,t}}"
                assert rhos[y*T2+t, x*T2+s] == rhos[x*T2+s, y*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{YX}}_{{t,s}} = \rho^{{XY}}_{{s,t}}"

In [7]:
rhos.T == rhos

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [8]:
rhos

array([[ 0.29698109,  0.03198337, -0.05212048, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03198337,  0.29698109,  0.03198337, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05212048,  0.03198337,  0.29698109, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.13562352,
         0.04408787, -0.03850037],
       [ 0.        ,  0.        ,  0.        , ...,  0.04408787,
         0.13562352,  0.04408787],
       [ 0.        ,  0.        ,  0.        , ..., -0.03850037,
         0.04408787,  0.13562352]])

In [9]:
rhos.shape

(2516, 2516)

In [10]:
pickle.dump(rhos, open("tmp/rhos.pkl", "wb"))